In [0]:
def getAdCampaignSv():
    return spark.read.table('ad_campaign_sv')

def getUserProfileSv():
    return spark.read.table('user_profile_sv')

def getStoreSv():
    return spark.read.table('store_sv')

#Analyze data for each campaign, date, hour, os_type to get all events with counts
def getEventCountByOSType():
    from pyspark.sql.functions import hour, to_date
    df_c = getAdCampaignSv()
    df_c.groupBy("campaign_id", to_date("event_time").alias("event_date"), hour("event_time").alias("event_hour"), "os_type")\
        .count()\
        .show()

#Analyze data for each campaign, date, hour, store_name to get all events with counts
def getEventCountByStore():
    from pyspark.sql.functions import hour, to_date, count
    df_c = getAdCampaignSv()
    df_s = getStoreSv()
    df_c.join(df_s, df_c.place_id == df_s.place_id, "left")\
        .groupBy(df_c.campaign_id, to_date(df_c.event_time).alias("event_date"), hour(df_c.event_time).alias("event_hour"), 
                df_s.store_name)\
        .agg(count(df_c.campaign_id).alias("count_events"))\
        .show()

#Analyze data for each campaign, date, hour, gender_type to get all events with counts
def getEventCountByGenderType():
    from pyspark.sql.functions import hour, to_date, count
    df_c = getAdCampaignSv()
    df_u = getUserProfileSv()
    df_c.join(df_u, df_c.user_id == df_u.user_id, "left")\
        .selectExpr("* except(category_expl)")\
        .distinct()\
        .groupBy(df_c.campaign_id, to_date(df_c.event_time).alias("event_date"), hour(df_c.event_time).alias("event_hour"), 
                df_u.gender)\
        .agg(count(df_c.campaign_id).alias("count_events"))\
        .show()

In [0]:
getEventCountByOSType()

In [0]:
getEventCountByStore()

In [0]:
getEventCountByGenderType()

Campaign, UserProfile and Store - Silver data -->

In [0]:
%sql
select * from ad_campaign_sv

In [0]:
%sql
select * from user_profile_sv

In [0]:
%sql
select * from store_sv